In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
savePath = "s3://corpus-2/model/cnn"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    contents, labels = [], []
    with open(filePath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            contents.append(content)
            labels.append(label)

    return contents, labels


def readCategory():
    """读取分类目录，固定id"""
    '''
    Retrieve Value
    Filter
    Compute Derived Value
    Find Extremum
    Sort
    Determine Range
    Characterize Distribution
    Find Anomalies
    Cluster
    Correlate
    '''
    categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
    cat_to_id = dict(zip(categories, range(1,len(categories)+1)))
    id_to_cat = dict(zip(range(1,len(categories)+1), categories))
    return id_to_cat, cat_to_id


def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd

In [4]:
contents_train, labels_train = readfile(trainDataPath)

In [5]:
len(contents_train),len(labels_train)

(14035, 14035)

In [6]:
id_to_cat, cat_to_id = readCategory()
num_classes = len(id_to_cat)

In [7]:
num_classes, labels_train[0]

(10, 1)

In [8]:
seq_length = 0
for content in contents_train:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 41

In [9]:
seq_length

41

In [10]:
vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loading GloVe!
Completed!


In [11]:
word_to_id = dict(zip(vocab, range(vocab_size)))

In [12]:
len(embedding),embedding_dim,vocab_size

(400000, 100, 400000)

In [13]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [14]:
print("Loading training and validation and testing data...")
start_time = time.time()
x_train, y_train = process_file(contents_train, labels_train, word_to_id, num_classes, seq_length)
time_dif = get_time_dif(start_time)
print("Loading data Time usage:", time_dif)

Loading training and validation and testing data...
Loading data Time usage: 0:00:00


In [15]:
x_train[0], y_train[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,  102,   14,    0,
         485,    3, 4791,   13,  375, 2693,  812,  188], dtype=int32),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [16]:
len(x_train)

14035

In [17]:
word_to_id["arkansas"]

4791

In [18]:
embedding[3]

array([-0.1529  , -0.24279 ,  0.89837 ,  0.16996 ,  0.53516 ,  0.48784 ,
       -0.58826 , -0.17982 , -1.3581  ,  0.42541 ,  0.15377 ,  0.24215 ,
        0.13474 ,  0.41193 ,  0.67043 , -0.56418 ,  0.42985 , -0.012183,
       -0.11677 ,  0.31781 ,  0.054177, -0.054273,  0.35516 , -0.30241 ,
        0.31434 , -0.33846 ,  0.71715 , -0.26855 , -0.15837 , -0.47467 ,
        0.051581, -0.33252 ,  0.15003 , -0.1299  , -0.54617 , -0.37843 ,
        0.64261 ,  0.82187 , -0.080006,  0.078479, -0.96976 , -0.57741 ,
        0.56491 , -0.39873 , -0.057099,  0.19743 ,  0.065706, -0.48092 ,
       -0.20125 , -0.40834 ,  0.39456 , -0.02642 , -0.11838 ,  1.012   ,
       -0.53171 , -2.7474  , -0.042981, -0.74849 ,  1.7574  ,  0.59085 ,
        0.04885 ,  0.78267 ,  0.38497 ,  0.42097 ,  0.67882 ,  0.10337 ,
        0.6328  , -0.026595,  0.58647 , -0.44332 ,  0.33057 , -0.12022 ,
       -0.55645 ,  0.073611,  0.20915 ,  0.43395 , -0.012761,  0.089874,
       -1.7991  ,  0.084808,  0.77112 ,  0.63105 , 

In [19]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']

In [20]:
class rnn_cnn(object):
    
    def __init__(self, savePath):
        self.savePath = savePath
        # 输入内容及对应的标签
        self.input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        # dropout的损失率
        self.keep_prob = tf.placeholder(tf.float64, name='keep_prob')
        # 词向量映射;实际上此处的词向量并不是用的预训练好的词向量，而是未经任何训练直接生成了一个矩阵，将此矩阵作为词向量矩阵使用，效果也还不错。
        # 若使用训练好的词向量，或许训练此次文本分类的模型时会更快，更好。
        # embedding = tf.get_variable('embedding', [vocab_size, embedding_dim])
        embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)

        num_filters = 256
        kernel_size = 5
        hidden_dim = 128
        learning_rate = 1e-3

        # CNN layer
        conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size, name='conv')  # num_filters = 256 这是个啥
        ''' https://blog.csdn.net/khy19940520/article/details/89934335
        tf.layers.conv1d：一维卷积一般用于处理文本数据，常用语自然语言处理中，输入一般是文本经过embedding的二维数据。
            inputs： 输入tensor， 维度(batch_size, seq_length, embedding_dim) 是一个三维的tensor；其中，
                batch_size指每次输入的文本数量；
                seq_length指每个文本的词语数或者单字数；
                embedding_dim指每个词语或者每个字的向量长度；
                例如每次训练输入2篇文本，每篇文本有100个词，每个词的向量长度为20，那input维度即为(2, 100, 20)。
            filters：过滤器（卷积核）的数目
            kernel_size：卷积核的大小，卷积核本身应该是二维的，这里只需要指定一维，因为第二个维度即长度与词向量的长度一致，卷积核只能从上往下走，不能从左往右走，即只能按照文本中词的顺序，也是列的顺序。
        '''
        # global max pooling layer
        gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')  # https://blog.csdn.net/lllxxq141592654/article/details/85345864

        # 全连接层，后面接dropout以及relu激活
        fc = tf.layers.dense(gmp, hidden_dim, name='fc1')  # hidden_dim：128
        ''' https://blog.csdn.net/yangfengling1023/article/details/81774580
        dense ：全连接层  inputs：输入该网络层的数据；units：输出的维度大小，改变inputs的最后一维
        '''
        fc = tf.nn.dropout(fc, self.keep_prob)
        fc = tf.nn.relu(fc)

        # 分类器
        logits = tf.layers.dense(fc, num_classes, name='fc2')
        self.y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

        # 损失函数，交叉熵
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.input_y)
        self.loss = tf.reduce_mean(cross_entropy)
        # 优化器
        self.optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.saver = tf.train.Saver()
        
    
    def train(self, x_train, y_train, split_type, fold_id, num_epochs=20, dropout_keep_prob=0.5, print_per_batch=30, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 创建session
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        
        print('Training and evaluating...')
        start_time = time.time()
        total_batch = 0  # 总批次
        best_acc_train = 0.0  # 最佳验证集准确率
        last_improved = 0  # 记录上一次提升批次
        require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
        flag = False

        for epoch in range(num_epochs):  # 20
            print('Epoch:', epoch + 1)
            batch_train = rnn_cnn.batch_iter(x_train, y_train, batch_size)
            for x_batch, y_batch in batch_train:
                feed_dict = {self.input_x: x_batch, self.input_y: y_batch, self.keep_prob: dropout_keep_prob}
                session.run(self.optim, feed_dict=feed_dict)  # 运行优化
                total_batch += 1

                if total_batch % print_per_batch == 0:
                    # 每多少轮次输出在训练集和验证集上的性能
                    feed_dict[self.keep_prob] = 1.0
                    loss_train, acc_train = session.run([self.loss, self.acc], feed_dict=feed_dict)
                    # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                    if acc_train > best_acc_train:
                        # 保存最好结果
                        best_acc_train = acc_train
                        last_improved = total_batch
                        self.saver.save(sess=session, save_path=savePath)
                        improved_str = '*'
                    else:
                        improved_str = ''

                    time_dif = get_time_dif(start_time)
                    msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                    print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

                if total_batch - last_improved > require_improvement:
                    # 验证集正确率长期不提升，提前结束训练
                    print("No optimization for a long time, auto-stopping...")
                    flag = True
                    break  # 跳出循环
            if flag:  # 同上
                break
                
        session.close()
        return best_acc_train
        
        
    def evaluate_model(self, x_test, y_test, split_type, fold_id, categories, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 读取保存的模型
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        start_time = time.time()
        print('Testing...')
        loss_test, acc_test = self.evaluate(session, x_test, y_test, self.loss, self.acc, batch_size)
        msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
        print(msg.format(loss_test, acc_test))

        test_data_len = len(x_test)
        test_num_batch = int((test_data_len - 1) / batch_size) + 1

        y_test_cls = np.argmax(y_test, 1)  # 获得类别
        y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

        for i in range(test_num_batch):  # 逐批次处理
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, test_data_len)
            feed_dict = {
                self.input_x: x_test[start_id:end_id],
                self.keep_prob: 1.0
            }
            y_test_pred_cls[start_id:end_id] = session.run(self.y_pred_cls, feed_dict=feed_dict)

        # 评估
        print("Precision, Recall and F1-Score...")
        print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
        '''
        sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
            y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
            y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
            labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
            target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
            原文链接：https://blog.csdn.net/akadiao/article/details/78788864
        '''

        # 混淆矩阵
        print("Confusion Matrix...")
        cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
        '''
        混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
        这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
        https://blog.csdn.net/u011734144/article/details/80277225
        '''
        print(cm)

        time_dif = get_time_dif(start_time)
        print("Time usage:", time_dif)
        session.close()
        
        return acc_test
        
        
    def predict(self, predict_sentences, word_to_id, pad_max_length, split_type, fold_id):
        """
        将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
        """
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        
        data_id = []
        # 将文本内容转换为对应的id形式
        for i in range(len(predict_sentences)):
            data_id.append([word_to_id[x] for x in predict_sentences[i].lower().strip().split() if x in word_to_id])

        # 使用keras提供的pad_sequences来将文本pad为固定长度
        x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
        ''' https://blog.csdn.net/TH_NUM/article/details/80904900
        pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
            sequences：浮点数或整数构成的两层嵌套列表
            maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
            dtype：返回的numpy array的数据类型
            padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
            truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
            value：浮点数，此值将在填充时代替默认的填充值0
        '''
        feed_dict = {
            self.input_x: x_pad,
            self.keep_prob: 1.0
        }
        predict_result = session.run(self.y_pred_cls, feed_dict=feed_dict)
        predict_result = [i+1 for i in predict_result]
        session.close()
        
        return predict_result
    
    
    def evaluate(self, sess, x_pad, y_pad, loss1, acc1, batch_size):
        """评估在某一数据上的准确率和损失"""
        data_len = len(x_pad)
        batch_eval = rnn_cnn.batch_iter(x_pad, y_pad, batch_size)  
        total_loss = 0.0
        total_acc = 0.0
        # print(dropout_keep_prob)
        for x_batch1, y_batch1 in batch_eval:
            batch_len = len(x_batch1)
            feed_dict1 = {self.input_x: x_batch1, self.input_y: y_batch1, self.keep_prob: 1.0}
            lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
            total_loss += lossTmp * batch_len
            total_acc += accTmp * batch_len

        return total_loss / data_len, total_acc / data_len
    
    
    def batch_iter(x_pad, y_pad, batch_size=64):  # 128
        """生成批次数据"""
        data_len = len(x_pad)
        num_batch = int((data_len - 1) / batch_size) + 1
        # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
        # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
        indices = np.random.permutation(np.arange(data_len))
        x_shuffle = x_pad[indices]
        y_shuffle = y_pad[indices]

        # 返回所有batch的数据
        for i in range(num_batch):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, data_len)
            yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
            
            

In [21]:
kf = KFold(n_splits=10)
fold_id = 0
model_train_acc = []
model_test_acc = []
model = rnn_cnn(savePath)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [22]:
split_type = "random"
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    model_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    model_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.0, Train Acc:  48.44%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.7, Train Acc:  51.56%, Time: 0:00:14 *
Iter:     90, Train Loss:    1.4, Train Acc:  57.81%, Time: 0:00:20 *
Iter:    120, Train Loss:    1.1, Train Acc:  71.88%, Time: 0:00:25 *
Iter:    150, Train Loss:   0.93, Train Acc:  73.44%, Time: 0:00:30 *
Iter:    180, Train Loss:    0.8, Train Acc:  71.88%, Time: 0:00:31 
Epoch: 2
Iter:    210, Train Loss:   0.57, Train Acc:  81.25%, Time: 0:00:36 *
Iter:    240, Train Loss:   0.57, Train Acc:  84.38%, Time: 0:00:41 *
Iter:    270, Train Loss:   0.77, Train Acc:  82.81%, Time: 0:00:42 
Iter:    300, Train Loss:   0.43, Train Acc:  87.50%, Time: 0:00:46 *
Iter:    330, Train Loss:   0.43, Train Acc:  89.06%, Time: 0:00:53 *
Iter:    360, Train Loss:   0.29, Train Acc:  93.75%, Time: 0:00:59 *
Iter:    390, Train Loss:   0.21, Train Acc:  96.88%, Time: 0:01:04 *
Epoch: 3
Iter:    420, Train Loss:   0

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn/random/1/1


Testing...
Test Loss:    1.4, Test Acc:  64.81%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.69      0.57      0.62       184
                   Filter       0.58      0.58      0.58       187
    Compute Derived Value       0.61      0.51      0.55       145
            Find Extremum       0.52      0.81      0.63       120
                     Sort       0.73      0.97      0.83       122
          Determine Range       0.81      0.85      0.83       131
Characterize Distribution       0.88      0.48      0.62       141
           Find Anomalies       0.52      0.75      0.62       134
                  Cluster       0.63      0.45      0.52       125
                Correlate       0.74      0.61      0.67       115

                micro avg       0.65      0.65      0.65      1404
                macro avg       0.67      0.66      0.65      1404
             weighted avg       0.67      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn/random/2/2


Testing...
Test Loss:    1.7, Test Acc:  53.92%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.77      0.26      0.39       179
                   Filter       0.21      0.16      0.18       168
    Compute Derived Value       0.75      0.41      0.53       284
            Find Extremum       0.51      0.74      0.61       167
                     Sort       0.72      0.87      0.79       122
          Determine Range       0.30      0.62      0.41        94
Characterize Distribution       0.89      0.42      0.57        97
           Find Anomalies       0.48      0.63      0.54        94
                  Cluster       0.62      0.90      0.74        84
                Correlate       0.54      0.89      0.67       115

                micro avg       0.54      0.54      0.54      1404
                macro avg       0.58      0.59      0.54      1404
             weighted avg       0.59      0.5

...

Testing...
Test Loss:    1.1, Test Acc:  70.49%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.72      0.58      0.64       116
                   Filter       0.64      0.62      0.63       141
    Compute Derived Value       0.44      0.67      0.53       123
            Find Extremum       0.69      0.94      0.79       124
                     Sort       0.85      0.70      0.77       122
          Determine Range       0.67      0.54      0.60       118
Characterize Distribution       0.93      0.84      0.88       164
           Find Anomalies       0.60      0.67      0.63       177
                  Cluster       0.85      0.78      0.81       168
                Correlate       0.82      0.67      0.74       150

                micro avg       0.70      0.70      0.70      1403
                macro avg       0.72      0.70      0.70      1403
             weighted avg       0.73      0.7

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn/random/9/9


Testing...
Test Loss:    1.3, Test Acc:  68.07%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.95      0.55      0.69       137
                   Filter       0.88      0.49      0.63       143
    Compute Derived Value       0.69      0.50      0.58       153
            Find Extremum       0.85      0.69      0.76       150
                     Sort       0.78      0.91      0.84       137
          Determine Range       0.81      0.61      0.70       158
Characterize Distribution       0.84      0.75      0.79       143
           Find Anomalies       0.77      0.59      0.67       121
                  Cluster       0.59      0.81      0.68       127
                Correlate       0.38      0.95      0.54       134

                micro avg       0.68      0.68      0.68      1403
                macro avg       0.75      0.68      0.69      1403
             weighted avg       0.76      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn/random/10/10


Testing...
Test Loss:    1.1, Test Acc:  64.65%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.51      0.45      0.48       128
                   Filter       0.55      0.48      0.51       131
    Compute Derived Value       0.37      0.36      0.36       125
            Find Extremum       0.70      0.91      0.79       128
                     Sort       0.90      0.69      0.78       112
          Determine Range       0.44      0.55      0.49       121
Characterize Distribution       0.71      0.91      0.80       138
           Find Anomalies       0.61      0.76      0.68       156
                  Cluster       0.94      0.62      0.74       165
                Correlate       0.79      0.68      0.73       199

                micro avg       0.65      0.65      0.65      1403
                macro avg       0.65      0.64      0.64      1403
             weighted avg       0.66      0.6

In [24]:
model_train_acc

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [23]:
model_test_acc

[0.6481481476387068,
 0.5391737900228581,
 0.5534188027395482,
 0.6673789165298484,
 0.695868944850063,
 0.6557377045781632,
 0.6015680691045433,
 0.704918032531983,
 0.6806842477000448,
 0.6464718451520332]

In [26]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)

(0.6393368500847791,
 0.05406597768875925,
 0.05699054447344181,
 0.0029231299434414135)